# Download images from the ACT website

In [2]:
import pandas as pd
import time
import requests
import math
from typing import List, Dict, Tuple, Any, Optional

from PIL import Image
from PIL.ExifTags import TAGS
import os
import datetime
import csv
import json
import exifread # https://github.com/ianare/exif-py

# Global variables
IMAGE_DOWNLOAD_DIRECTORY = '/Users/baskausj/Downloads/act_images/'
CLEAN_DATA_DIRECTORY = '/Users/baskausj/github/vandycite/act/processed_lists/'
ACT_BASE_URL = 'https://diglib.library.vanderbilt.edu/act-imagelink.pl?RC='
YEARS_SINCE_INCEPTION_SCREENING_AGE = 150

# Functions
def csv_read(path: str, **kwargs) -> pd.DataFrame:
    """Loads a CSV table into a Pandas DataFrame with all cells as strings and blank cells as empty strings
    
    Keyword argument:
    rows -- the number of rows of the table to return when used for testing. When omitted, all rows are returned.
    """
    dataframe = pd.read_csv(path, na_filter=False, dtype = str)
    if 'rows' in kwargs:
        return dataframe.head(kwargs['rows']).copy(deep=True)
    else:
        return dataframe

def generate_utc_date():
    whole_time_string_z = datetime.datetime.utcnow().isoformat() # form: 2019-12-05T15:35:04.959311
    date_z = whole_time_string_z.split('T')[0] # form 2019-12-05
    return date_z

today = generate_utc_date()

# From https://github.com/HeardLibrary/vandycite/blob/master/act/create_items/create_act_items.ipynb
def pad_zeros_left(date_string):
    length = len(date_string)
    pad = 4-length
    return '0' * pad + date_string

def generate_date_string(date, bce):
    if bce:
        date_string = '-'
    else:
        date_string = ''
    date_string += pad_zeros_left(str(date)) + '-01-01T00:00:00Z'
    return date_string
 
# Parse the ACT date string into structured components
# From https://github.com/HeardLibrary/vandycite/blob/master/act/create_items/create_act_items.ipynb
def process_act_date(act_date):
    act_circa = False
    act_range = False
    act_century = False
    non_numeric = False
    date = 0
    start_date = 0
    end_date = 0
    
    # If there is no date from ACT, kill the function and return False
    if act_date == '':
        return False, date, act_range, start_date, end_date, act_century, act_circa
    
    # Determine circa status of ACT date
    if 'ca.' in act_date:
        act_circa = True
        # Remove the "ca." from the beginning and clean whitespace
        act_date = act_date.split('ca.')[1].strip()
    
    # Test whether the ACT date is a number
    try:
        date = int(act_date)
        #print('numeric date:', date)
    except:
        non_numeric = True
        #print('non-numeric string:', act_date)
        
    if non_numeric:
        # Determine century status of ACT date
        if 'century' in act_date: # single century date
            act_century = True
            # Remove the "century" and "th", "rd", "st", etc. from the end
            act_date = act_date[:-10]
            non_numeric = False
            try:
                date = int(act_date) * 100 - 50 # set the date at mid-century
            except:
                print('numeric conversion error on', act_date)
        elif 'centuries' in act_date:
            act_century = True
            act_range = True
            # Remove the "centuries" and "th", "rd", "st", etc. from the end
            act_date = act_date[:-10].strip()
            try:
                pieces = act_date.split('-')
                start_date = int(pieces[0][:-2]) * 100 - 50 # set the date at mid-century
                end_date = int(pieces[1][:-2]) * 100 - 50 # set the date at mid-century
            except:
                print('error in processing century range')
    # Process date ranges (non-numeric because they include "-")
    if non_numeric and not act_century:
        #print(act_date)
        try:
            pieces = act_date.split('-')
            start_date = int(pieces[0])
            end_date = int(pieces[1])
            act_range = True
        except:
            print('error in processing date range')

        
    # if there is a range of dates, set the single date as the midpoint
    if start_date != 0 or end_date != 0:
        date = math.floor((start_date + end_date)/2)
            
    return True, date, act_range, start_date, end_date, act_century, act_circa
 

In [ ]:
# Read in clean metadata CSV as text into dataframe, with NA values as empty strings
metadata_path = '../processed_lists/clean_metadata_2022-09-29.csv'
act_metadata_df = pd.read_csv(metadata_path, dtype=str, na_filter=False)

# Find all rows whose lower case "CopyrightStatement" column value equals "image donated by Jim Womack and Anne Richardson" or variant
richardson_df1 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'image donated by Jim Womack and Anne Richardson']
richardson_df2 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'Image donated by Jim Womack and Anne Richardson']
richardson_df3 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'Image donated by Anne Richardson']
# Combine the two dataframes
richardson_df = pd.concat([richardson_df1, richardson_df2, richardson_df3])

# Set the RecordNumber column as the index
richardson_df.set_index('RecordNumber', inplace=True)

# Save the slice as a CSV
#richardson_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'richardson.csv')

# Slice the values from the ImageLink column and create a new dataframe with them, using the same index as the original dataframe
filenames_df = richardson_df.loc[:, 'ImageLink']
filenames_df = pd.DataFrame(filenames_df)

# Add a column named file_url
filenames_df['file_url'] = ''

# Save the slice as a CSV
filenames_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv')

filenames_df.head()


In [ ]:
# In case a script crash requires restarting, reload the CSV file
filenames_df = pd.read_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv', dtype=str, na_filter=False)

# Set the index to the RecordNumber column
filenames_df = filenames_df.set_index('RecordNumber')

# Loop through the rows and download the images from the URL
for index, row in filenames_df.iterrows():
    # If the file_url column is not empty, skip this row
    if row['file_url'] != '':
        continue

    # Convert the filename to a string
    filename_string = row['ImageLink']

    # If the filename contains only numeric characters, padd it on the left with 0s until it has 8 characters
    if filename_string.isnumeric():
        filename_string = filename_string.zfill(8)
    url = 'https://diglib.library.vanderbilt.edu/cdri/fulljpeg/' + filename_string + '.jpg'
    print(url)
    
    # Add the file URL to the dataframe
    filenames_df.at[index, 'file_url'] = url

    # Download the image
    filename = filename_string + '.jpg'
    r = requests.get(url)
    with open(IMAGE_DOWNLOAD_DIRECTORY + filename, 'wb') as outfile:
        outfile.write(r.content)

    # Wait 0.5 second before downloading the next image
    time.sleep(0.5)

    # Write the dataframe to a CSV file after each download
    # to keep track of which ones were done in case the script crashes.
    filenames_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv')
    

# Build the images.csv file needed for input into CommonsTool

In some cases, the file download failed due to a 404. These files have dimensions of 0,0 and kilobyte values of 1.

After completing this step, I had to manually indicate whether it was 3D or 2D when creating the artwork_metadata.csv file. It can be minimally created by copying and pasting columns from the images.csv file. In the case of Anne's images, most were 3D, so I just needed to mark the 2D ones and fill in 3D for the rest.

In [ ]:
# Create dataframe to build the images.csv table.       
images_df = csv_read(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv') # add rows keyword to limit number of rows read
#images_df = csv_read(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv', rows=2) # add rows keyword to limit number of rows read

# Rename the RecordNumber to local_identifier
images_df = images_df.rename(columns={'RecordNumber': 'local_identifier'})

# Set the index to the RecordNumber column
images_df = images_df.set_index('local_identifier')

# Extract the file name from the file_url column.
images_df['local_filename'] = images_df['file_url'].apply(lambda x: x.split('/')[-1])

# Remove the ImageLink and file_url columns.
images_df = images_df.drop(['ImageLink', 'file_url'], axis=1)

# Code hacked from https://github.com/HeardLibrary/linked-data/blob/master/commonsbot/extract_image_metadata.ipynb

for index,image_row in images_df.iterrows():
    image_name = image_row['local_filename']
    if image_name[0] == '.': # skip hidden files
        continue

    extension = image_name.split('.')[-1] # separate into pieces by full stops and take the last piece
    image_path = IMAGE_DOWNLOAD_DIRECTORY + image_name
    # trap errors when the file isn't an image
    try:
        with Image.open(image_path) as img:
            width, height = img.size
    except:
        width = 0
        height = 0
        
    try:
        # First try to get the actual image creation date from the EXIF
        # Code from https://stackoverflow.com/questions/23064549/get-date-and-time-when-photo-was-taken-from-exif-data-using-pil
        with open(image_path, 'rb') as fh:
            tags = exifread.process_file(fh, stop_tag='EXIF DateTimeOriginal')
            date_taken = tags['EXIF DateTimeOriginal']
            create_date_string = str(date_taken)[:10].replace(':', '-')
            #print('EXIF DateTimeOriginal', create_date_string)
            if create_date_string == '0000-00-00':
                raise Exception('Bad date')
            #print('image date')
    except:
        # If that's unavailable, then use the file creation date.
        # Note: this code is Mac/Linux-specific and would need to be modified if run on Windows.
        timestamp = os.stat(image_path).st_birthtime
        time_object = datetime.datetime.fromtimestamp(timestamp)
        create_date_string = time_object.strftime("%Y-%m-%d")
        #print('file date', create_date_string)
        

    if create_date_string == '1969-12-31':
        timestamp = os.stat(image_path).st_mtime 
        time_object = datetime.datetime.fromtimestamp(timestamp)
        create_date_string = time_object.strftime("%Y-%m-%d")
        #print('file modified', create_date_string)

    #print(image_path, create_date_string)

    #print(height, width)
    #print()

    # Set the values in the current row of the dataframe
    images_df.loc[index, 'qid'] = ''
    images_df.loc[index, 'rank'] = 'primary'
    images_df.loc[index, 'label'] = '' 
    images_df.loc[index, 'notes'] = '' 
    images_df.loc[index, 'kilobytes'] = round(os.path.getsize(image_path)/1024)
    images_df.loc[index, 'height'] = height
    images_df.loc[index, 'width'] = width
    images_df.loc[index, 'photo_inception'] = create_date_string
    images_df.loc[index, 'subdir'] = ''

# Unset the index
images_df = images_df.reset_index()

# Move qid to the first column in the dataframe
qid_column = images_df.pop('qid')
images_df.insert(0, 'qid', qid_column)

# Move local_identifier to the third column in the dataframe
local_identifier_column = images_df.pop('local_identifier')
images_df.insert(2, 'local_identifier', local_identifier_column)

# Save the dataframe to a CSV file in the same directory as this notebook
images_df.to_csv('images.csv', index=False)
print('done')


# Create the Wikidata input file

In the test, this was called `act_artworks.csv` and it's column headers were mapped in the `config.csv` file. The fields need to be filled from the ACT download file, filtered for Anne's images (`richardson.csv`).

Note: before this step, I added a column to the images.csv file called `dimension` to record whether it was a 3D or 2D work. This value needs to be in the `richardson.csv` file in order to do the Commons upload, but since I needed to sort through the images to see which ones were black and white or otherwise unusable anyway, this was the easiest way to to get the value and I can transfer it over to the `richardson.csv` source data file from the `images.csv` file since there is a one-to-one correspondence between the images and the artwork items anyway. Both spreadsheets will have `dimension` columns.

In [ ]:
# Code to transfer dimension column from images.csv to richardson.csv by matching the RecordNumber column.

# Read in the images.csv file
images_df = csv_read('images.csv')

# Set the index to the local_identifier column
images_df = images_df.set_index('local_identifier')

# Read in the richardson.csv file
richardson_df = csv_read('richardson.csv')

# Set the index to the RecordNumber column
richardson_df = richardson_df.set_index('RecordNumber')

# Loop through the rows of the richardson.csv file
for index, row in richardson_df.iterrows():
    # If the RecordNumber column is empty, skip this row
    if index == '':
        continue

    # If the RecordNumber column is not empty, get the value from the images.csv file
    try:
        dimension = images_df.loc[index, 'dimension']
    except:
        dimension = ''
    
    # Set the value in the dimension column
    richardson_df.at[index, 'dimension'] = dimension

# Unset the index
richardson_df = richardson_df.reset_index()

# Move the RecordNumber column to the first column in the dataframe
record_number_column = richardson_df.pop('RecordNumber')
richardson_df.insert(0, 'RecordNumber', record_number_column)

# Save the dataframe to a CSV file in the same directory as this notebook
richardson_df.to_csv('richardson.csv', index=False)
print('done')


Have to modify the original data to remove the duplicates Charlotte identified and to replace rows with the cleaned data. 

Note: At this stage, I manually added the `part_of` column to the richardson.csv file since we had added it to the duplicate file. But none of the works in the original file have values for part_of, so we should probably go back later and add it.

In [ ]:
# Get the raw data from the ACT database export. Modify it using cleaned data.     
raw_df = csv_read('richardson.csv') # add rows keyword to limit number of rows read
# Set the RecordNumber (ACT ID) column as the index
raw_df = raw_df.set_index('RecordNumber')

# Create a list of the works to skip
redundant_works_to_skip = csv_read('redundant_items.csv')
redundant_works_list = redundant_works_to_skip['RecordNumber'].tolist()

# These are the records that need to be replaced in the raw_df
duplicate_works_metadata = csv_read('richardson_duplicate_titles_prep.csv')
# Set the RecordNumber (ACT ID) column as the index
duplicate_works_metadata = duplicate_works_metadata.set_index('RecordNumber')

# Loop through the rows of the raw_df
for index, row in raw_df.iterrows():
    # If the index is in the redundant_works_list, remove it from the dataframe
    if index in redundant_works_list:
        raw_df.drop(index, inplace=True)
        continue

# Loop through the rows of the duplicate_works_metadata. 
# Replace the corresponding row in the raw_df with the row from the duplicate_works_metadata.
for index, row in duplicate_works_metadata.iterrows():
    raw_df.loc[index] = duplicate_works_metadata.loc[index]

# Save the raw_df as a CSV file
raw_df.to_csv('richardson.csv', index=True)
print('done')


Now actually create the output file

In [ ]:
# Get the raw data from the ACT database export. Modify it using cleaned data.     
raw_df = csv_read('richardson.csv') # add rows keyword to limit number of rows read
#raw_df = csv_read('richardson.csv', rows=2) # add rows keyword to limit number of rows read

# Open the images.csv file and read it into a dataframe.
images_df = csv_read('images.csv')

country_mappings = pd.read_csv('country_mappings.csv', na_filter=False, dtype = str)

# Create a list of the column names for the output CSV file.
column_list = [
    'qid',
    'label_en',
    'description_en',
    'act_uuid',
    'act',
    'act_ref1_hash',
    'act_ref1_retrieved_nodeId',
    'act_ref1_retrieved_val',
    'act_ref1_retrieved_prec',
    'inventory_number_uuid',
    'inventory_number',
    'inventory_number_collection',
    'inventory_number_ref1_hash',
    'inventory_number_ref1_referenceUrl',
    'inventory_number_ref1_retrieved_nodeId',
    'inventory_number_ref1_retrieved_val',
    'inventory_number_ref1_retrieved_prec',
    'title_uuid',
    'title',
    'title_ref1_hash',
    'title_ref1_referenceUrl',
    'title_ref1_retrieved_nodeId',
    'title_ref1_retrieved_val',
    'title_ref1_retrieved_prec',
    'creator_uuid',
    'creator',
    'creator_object_has_role',
    'creator_ref1_hash',
    'creator_ref1_referenceUrl',
    'creator_ref1_retrieved_nodeId',
    'creator_ref1_retrieved_val',
    'creator_ref1_retrieved_prec',
    'instance_of_uuid',
    'instance_of',
    'instance_of_ref1_hash',
    'instance_of_ref1_referenceUrl',
    'instance_of_ref1_retrieved_nodeId',
    'instance_of_ref1_retrieved_val',
    'instance_of_ref1_retrieved_prec',
    'inception_uuid',
    'inception',
    'inception_nodeId',
    'inception_val',
    'inception_prec',
    'inception_earliest_date_nodeId',
    'inception_earliest_date_val',
    'inception_earliest_date_prec',
    'inception_latest_date_nodeId',
    'inception_latest_date_val',
    'inception_latest_date_prec',
    'inception_sourcing_circumstances',
    'inception_ref1_hash',
    'inception_ref1_referenceUrl',
    'inception_ref1_retrieved_nodeId',
    'inception_ref1_retrieved_val',
    'inception_ref1_retrieved_prec',
    'country_of_origin_uuid',
    'country_of_origin',
    'country_of_origin_ref1_hash',
    'country_of_origin_ref1_referenceUrl',
    'country_of_origin_ref1_retrieved_nodeId',
    'country_of_origin_ref1_retrieved_val',
    'country_of_origin_ref1_retrieved_prec',
    'copyright_status_uuid',
    'copyright_status',
    'copyright_status_applies_to_jurisdiction',
    'copyright_status_determination_method',
    'copyright_status_ref1_hash',
    'copyright_status_ref1_referenceUrl',
    'copyright_status_ref1_retrieved_nodeId',
    'copyright_status_ref1_retrieved_val',
    'copyright_status_ref1_retrieved_prec',
    'image_uuid',
    'image',
    'image_ref1_hash',
    'image_ref1_referenceUrl',
    'image_ref1_retrieved_nodeId',
    'image_ref1_retrieved_val',
    'image_ref1_retrieved_prec',
    'collection_uuid',
    'collection',
    'collection_ref1_hash',
    'collection_ref1_referenceUrl',
    'collection_ref1_retrieved_nodeId',
    'collection_ref1_retrieved_val',
    'collection_ref1_retrieved_prec',
    'iiif_manifest_uuid',
    'iiif_manifest',
    'iiif_manifest_ref1_hash',
    'iiif_manifest_ref1_referenceUrl',
    'iiif_manifest_ref1_retrieved_nodeId',
    'iiif_manifest_ref1_retrieved_val',
    'iiif_manifest_ref1_retrieved_prec',
    'part_of_uuid',
    'part_of',
    'part_of_ref1_hash',
    'part_of_ref1_referenceUrl',
    'part_of_ref1_retrieved_nodeId',
    'part_of_ref1_retrieved_val',
    'part_of_ref1_retrieved_prec'
]

property_list = [
    'act',
    'inventory_number',
    'title',
    'creator',
    'instance_of',
    'inception',
    'country_of_origin',
    'copyright_status',
    'image',
    'collection',
    'iiif_manifest',
    'part_of'
]


# Create a new dataframe with the columns in the order specified in column_list.
act_artworks_df = pd.DataFrame(columns=column_list)

# Step through the rows in the raw dataframe.
for index, row in raw_df.iterrows():
    # Create a dict using the column names as keys with empty strings as values.
    act_artworks_dict = dict.fromkeys(column_list, '')

    # Add the values from the raw dataframe to the dict.

    # Check if the length of the label or description is greater than 250 characters.
    # If so, truncate the string. This is a hard limit imposed by Wikidata.
    if len(row['Title']) > 250:
        act_artworks_dict['label_en'] = row['Title'][:250]
        print(row['RecordNumber'], 'Warning: label truncated to 250 characters')
        print(row['Title'][:250])
        print()
    else:
        act_artworks_dict['label_en'] = row['Title']
    if row['ObjectFunction'] != '':
        act_artworks_dict['description_en'] = row['ObjectFunction'].lower() + ' ACT ID: ' + row['RecordNumber']
    else:
        act_artworks_dict['description_en'] = 'artwork ACT ID: ' + row['RecordNumber']
    
    # Screen for works without a DateCreation value
    if row['DateCreation'] == '':
        print(row['RecordNumber'], 'Warning: no DateCreation value')
        print()
        continue

    # Skip works whose image download failed.
    # These are detectable because their kilobytes value in the image_df is 1.
    image_match = images_df.loc[images_df['local_identifier'] == row['RecordNumber']]
    if len(image_match) == 0:
        print(row['RecordNumber'], 'Warning: skipping, no image found')
        print()
        continue

    if image_match['kilobytes'].values[0] == '1':
        print(row['RecordNumber'], 'Warning: skipping, image download failed')
        print()
        continue

    act_artworks_dict['act'] = row['RecordNumber']
    act_artworks_dict['title'] = row['Title']
    act_artworks_dict['creator'] = '_:' # Assume anonymous creator and change manually later if necessary.
    act_artworks_dict['creator_object_has_role'] = 'Q4233718'
    act_artworks_dict['instance_of'] = 'Q838948' # Assume "work of art" and change manually later if necessary.

    # Process dates.
    act_bce = False
    act_found, act_date, act_range, act_start_date, act_end_date, act_century, act_circa = process_act_date(row['DateCreation'])

    # Screen for works whose date is less than 200 years old (2023-200 = 1823)
    
    if act_date > 1823:
        print(row['RecordNumber'], 'Warning: date is less than 200 years old')
        print()
        continue

    if not act_found:
        act_artworks_dict['inception_val'] = ''
        act_artworks_dict['inception_sourcing_circumstances'] = ''
        act_artworks_dict['inception_prec'] = ''
        act_artworks_dict['inception_earliest_date_val'] = ''
        act_artworks_dict['inception_earliest_date_prec'] = ''
        act_artworks_dict['inception_latest_date_val'] = ''
        act_artworks_dict['inception_latest_date_prec'] = ''
        
    if act_found:
        act_artworks_dict['inception_val'] = generate_date_string(act_date, act_bce)
        if act_circa:
            act_artworks_dict['inception_sourcing_circumstances'] = 'Q5727902'
        else:
            act_artworks_dict['inception_sourcing_circumstances'] = ''
        if act_century:
            act_artworks_dict['inception_prec'] = '7'
        else:
            act_artworks_dict['inception_prec'] = '9'
        if act_range:
            act_artworks_dict['inception_earliest_date_val'] = generate_date_string(act_start_date, act_bce)
            if act_century:
                act_artworks_dict['inception_earliest_date_prec'] = '7'
            else:
                act_artworks_dict['inception_earliest_date_prec'] = '9'
            act_artworks_dict['inception_latest_date_val'] = generate_date_string(act_end_date, act_bce)
            if act_century:
                act_artworks_dict['inception_latest_date_prec'] = '7'
            else:
                act_artworks_dict['inception_latest_date_prec'] = '9'
        else:
            act_artworks_dict['inception_earliest_date_val'] = ''
            act_artworks_dict['inception_earliest_date_prec'] = ''
            act_artworks_dict['inception_latest_date_val'] = ''
            act_artworks_dict['inception_latest_date_prec'] = ''

    # Process country of origin
    country_string = row['LocationCountry'].strip()
    if row['OriginalLocation'] != '':
        country_string = row['OriginalLocation'].strip() # override if there is an original location (rare)
    if country_string != '':
        country_qid_series = country_mappings.loc[country_mappings.string == country_string, 'qid']
        if len(country_qid_series) == 1: # must be at least one match
            act_artworks_dict['country_of_origin'] = country_qid_series.values[0]
        else:
            act_artworks_dict['country_of_origin'] = ''

    # *** Find out if the work is Public Domain ***
    
    act_copyright_string = row['CopyrightStatus'].strip()
    
    # Start off with no values and overwrite as discovered
    act_artworks_dict['copyright_status'] = ''
    act_artworks_dict['copyright_status_applies_to_jurisdiction'] = ''
    act_artworks_dict['copyright_status_determination_method'] = ''

    # Determine years since inception date, if known
    if act_artworks_dict['inception_val'] != '':
        try:
            years_since_inception = int(today[:4]) - int(act_artworks_dict['inception_val'][:4])
        except:
            years_since_inception = 0
        if years_since_inception > YEARS_SINCE_INCEPTION_SCREENING_AGE:
            act_artworks_dict['copyright_status'] = 'Q19652' # Public Domain
            act_artworks_dict['copyright_status_applies_to_jurisdiction'] = 'Q60332278' # countries with 100 years pma or shorter
            act_artworks_dict['copyright_status_determination_method'] = 'Q29940705' # 100 years or more after author's death
        
    if 'public domain' in row['CopyrightStatus'].lower():
        # Determination method will be left blank since we don't know how the sources decided this
        act_artworks_dict['copyright_status'] = 'Q19652' # OK to write over value if already determined from dates

    # Add part_of data
    act_artworks_dict['part_of'] = row['part_of']

    # Add references for properties that have values.
    for property in property_list:
        if act_artworks_dict[property] != '':
            act_artworks_dict[property + '_ref1_referenceUrl'] = ACT_BASE_URL + act_artworks_dict['act']
            act_artworks_dict[property + '_ref1_retrieved_val'] = today

    # Remove the act_ref1_referenceUrl column since the ACT ID will be linked to a URL
    act_artworks_dict.pop('act_ref1_referenceUrl')

    # Add the dict to the dataframe.
    act_artworks_df = act_artworks_df.append(act_artworks_dict, ignore_index=True)

# Write the new dataframe to a CSV file.
act_artworks_df.to_csv('act_artworks.csv', index=False)

print('done')

# Processing titles

This is a one-time manipulation to sort out unique title/description combinations

In [ ]:
import pandas as pd

# Read in the richardson.xlsx file
richardson_df = pd.read_excel('richardson.xlsx', dtype=str, na_filter=False)
richardson_df.head()

# Create a dataframe to hold the rows with unique values in the Title column.
# Start by creating a blank dataframe with the same column headers as the richardson_df dataframe.
unique_titles_df = pd.DataFrame(columns=richardson_df.columns)

# Create another dataframe for the rows with duplicate values in the Title column.
# Start by creating a blank dataframe with the same column headers as the richardson_df dataframe.
duplicate_titles_df = pd.DataFrame(columns=richardson_df.columns)

# Create a list to hold the titles from the Title column.
title_list = richardson_df['Title'].tolist()

# Loop through the rows in the richardson_df dataframe.
for index, row in richardson_df.iterrows():
    # Get the title from the Title column.
    title = row['Title']

    # Determine if the title is in the title_list more than once.
    if title_list.count(title) > 1:
        # If the title is in the title_list more than once, add the row to the duplicate_titles_df dataframe.
        duplicate_titles_df = duplicate_titles_df.append(row, ignore_index=True)
    else:
        # If the title is in the title_list only once, add the row to the unique_titles_df dataframe.
        unique_titles_df = unique_titles_df.append(row, ignore_index=True)

# Write the unique_titles_df dataframe to a Excel file.
unique_titles_df.to_excel('richardson_unique_titles.xlsx', index=False)

# Write the duplicate_titles_df dataframe to a Excel file.
duplicate_titles_df.to_excel('richardson_duplicate_titles.xlsx', index=False)

print('done')


Step to generate the artwork_metadata.csv file needed as input for the commonstool.py script.

In [4]:
import pandas as pd

# Open the cleaned richardson.csv file (final cleaned up metadata dump) and read it into a dataframe.
richardson_df = csv_read('richardson.csv', na_filter=False, dtype = str)

# Open the act_artworks.csv file (VanderBot input/output file) and read it into a dataframe. (Needed to get the Q IDs that correspond to the ACT IDs.)
act_artworks_df = csv_read('act_artworks.csv', na_filter=False, dtype = str)

# Create a list of the column names for the output CSV file.
output_column_list = [
    'qid',
    'label_en',
    'act_id',
    'dimension',
    'status',
    'inception_val'
]

# Create a new dataframe with the columns in the order specified in column_list.
artwork_metadata_df = pd.DataFrame(columns=output_column_list)

# Copy the qid, label_en, inception_val, and act columns from the act_artworks_df dataframe to the artwork_metadata_df dataframe.
artwork_metadata_df['qid'] = act_artworks_df['qid']
artwork_metadata_df['label_en'] = act_artworks_df['label_en']
artwork_metadata_df['act_id'] = act_artworks_df['act']
artwork_metadata_df['inception_val'] = act_artworks_df['inception_val']

# Loop through each row in the artwork_metadata_df dataframe and add the dimension for the row whose RecordNumber value matches the act_id value in the artwork_metadata_df row.
for index, row in artwork_metadata_df.iterrows():
    # Get the act_id value from the artwork_metadata_df row.
    act_id = row['act_id']

    # Get the dimension value from the richardson_df row whose RecordNumber value matches the act_id value.
    dimension = richardson_df.loc[richardson_df['RecordNumber'] == act_id, 'dimension'].values[0]

    # Add the dimension value to the artwork_metadata_df row.
    artwork_metadata_df.at[index, 'dimension'] = dimension

# Write the artwork_metadata_df dataframe to a CSV file.
artwork_metadata_df.to_csv('artwork_metadata.csv', index=False)


Add data to images.csv file. After running and checking the output, I saved a copy of the original images.csv file as images_unscreened.csv and renamed images_out.csv to images.csv .

In [6]:
import pandas as pd

# Open dataframe containing the local_filename/local_identifier data and read it into a dataframe.
images_df = csv_read('images.csv', na_filter=False, dtype = str)

# Open the artwork_metadata.csv file and read it into a dataframe.
artwork_metadata_df = csv_read('artwork_metadata.csv', na_filter=False, dtype = str)

# Populate the qid and label column of the images_df dataframe with the Q IDs from the qid column and label_en column of the artwork_metadata_df by matching the local_identifier column of the images_df with the act_id column of the artwork_metadata_df.
for index, row in artwork_metadata_df.iterrows():
    # Get the act_id value from the artwork_metadata_df row.
    act_id = row['act_id']

    # Find the row in the images_df dataframe whose local_identifier value matches the act_id value and get the index of that row.
    images_df_index = images_df.index[images_df['local_identifier'] == act_id].tolist()[0]

    # Set the qid value in the images_df row to the qid value from the artwork_metadata_df row.
    images_df.at[images_df_index, 'qid'] = row['qid']

    # Set the label value in the images_df row to the label_en value from the artwork_metadata_df row.
    images_df.at[images_df_index, 'label'] = row['label_en']

# Remove all of the rows that do not have a qid value from the dataframe.
images_df = images_df[images_df['qid'] != '']

# Save the images_df dataframe to a CSV file.
images_df.to_csv('images_out.csv', index=False)


# Fix Macklin Bible artists

The artists were originally put in as anonymous, but the ACT website actually has all of their names. So they all need to be changed to the actual artist names.

The first step is to delete all of the existing anonymous creator statements. The output file is used with vanderdeletebot.py to delete the statements.

In [6]:
# Open the macklin.csv file of artist names and read it into a dataframe.
macklin_df = csv_read('macklin.csv', na_filter=False, dtype = str)
# Set the qid column as the index
macklin_df = macklin_df.set_index('qid')

# Open the act_artworks.csv file (VanderBot input/output file) and read it into a dataframe.
act_artworks_df = csv_read('act_artworks.csv', na_filter=False, dtype = str)
# Set the qid column as the index
act_artworks_df = act_artworks_df.set_index('qid')

# Create an empty dataframe to hold the rows whose artist needs to be deleted.
delete_artist_df = pd.DataFrame(columns=['qid', 'creator_uuid'])

# Loop through each row in the Macklin dataframe to get the qid. 
# Find the row in the act_artworks_df dataframe that has the same qid. 
# Add the qid and the creator_uuid as a new row to the delete_artist_df dataframe.
for index, row in macklin_df.iterrows():
    # Get the qid from the Macklin dataframe row index.
    qid = index

    # Get the creator_uuid from the act_artworks_df dataframe row whose qid matches the qid from the Macklin dataframe row.
    creator_uuid = act_artworks_df.loc[qid, 'creator_uuid']

    # Add the qid and the creator_uuid as a new row to the delete_artist_df dataframe.
    delete_artist_df = delete_artist_df.append({'qid': qid, 'creator_uuid': creator_uuid}, ignore_index=True)

# Save the delete_artist_df dataframe to a CSV file.
delete_artist_df.to_csv('deletions.csv', index=False)

Delete any rows in the spreadsheet where the artists weren't found.

The next step is to add change the creator UUID identifiers from the act_artworks.csv file to empty strings, and replace the creator value with the actual Q ID for the first artist of each work. NOTE: the creator_object_has_role column, creator ref1_hash column, and creator_ref1_retrieved_nodeId column also need to be cleared. 

In [13]:
# Open the macklin.csv file of artist names and read it into a dataframe.
macklin_df = csv_read('macklin.csv', na_filter=False, dtype = str)
# Set the qid column as the index
macklin_df = macklin_df.set_index('qid')

# Open the act_artworks.csv file (VanderBot input/output file) and read it into a dataframe.
act_artworks_df = csv_read('act_artworks.csv', na_filter=False, dtype = str)
# Set the qid column as the index
act_artworks_df = act_artworks_df.set_index('qid')

# Step through each row of the main dataframe.
for index, row in macklin_df.iterrows():
    # Find the row in the act_artworks_df dataframe that has the same index as the macklin_df row index.
    qid = index

    # Get the creator value from the macklin_df row.
    creator = row['artist1_qid']

    # Set the value of the creator_uuid, creator_object_has_role, creator_ref1_hash, 
    # and creator_ref1_retrieved_nodeId columns in the act_artworks_df row to empty string
    act_artworks_df.at[qid, 'creator_uuid'] = ''
    act_artworks_df.at[qid, 'creator_object_has_role'] = ''
    act_artworks_df.at[qid, 'creator_ref1_hash'] = ''
    act_artworks_df.at[qid, 'creator_ref1_retrieved_nodeId'] = ''

    # Set the value of the creator column in the act_artworks_df row to the creator value from the macklin_df row.
    act_artworks_df.at[qid, 'creator'] = creator

# Save the modified act_artworks_df dataframe to a CSV file.
act_artworks_df.to_csv('act_artworks.csv')

print('done')

done


Run Vanderbot to update the first artist added to the creator field.

Manually used the creator_additional.csv file and csv-metadata.json files from the gallery to add the second artists (sort and manual copy).

